In [2]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """
    Returns the Black–Scholes call price and Delta for a European call option.
    """
    # Avoid log(0) if S=0 or K=0 in edge cases
    S = np.array(S, dtype=float)
    K = np.array(K, dtype=float)
    T = np.array(T, dtype=float)
    r = np.array(r, dtype=float)
    sigma = np.array(sigma, dtype=float)

    sqrtT = np.sqrt(T)

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT

    # Call price
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    # Delta for a call is Phi(d1)
    delta = norm.cdf(d1)

    # Returns the bond position B that, combined with Delta shares of S, replicates the call option
    b_replicate = call_price - delta * S

    return call_price, delta, b_replicate


S = 100.0   # Underlying price
K = 100.0   # Strike price
T = 1.0     # 1 year to maturity
r = 0.05    # risk-free rate
sigma = 0.2 # volatility

call_price, delta, bond_position = black_scholes_call(S, K, T, r, sigma)
print("Call Price:", call_price)
print("Delta:", delta)

if bond_position >= 0:
    print(f"Bond position (B): {bond_position} (Invest in risk-free bond)")
else:
    print(f"Bond position (B): {bond_position} (Borrow at risk-free rate)")


Call Price: 10.450583572185565
Delta: 0.6368306511756191
Bond position (B): -53.232481545376345 (Borrow at risk-free rate)


In [24]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S: float, K: float, T: float, r: float, sigma: float) -> tuple[float, float]:
    """
    Returns the Black–Scholes call price and Delta for a European call option.

    Parameters
    ----------
    S: float Current stock price.
    K: float Strike price.
    T: float Time to maturity (in years).
    r: floatRisk-free interest rate (annualized).
    sigma: float Volatility of the underlying (annualized).

    Returns
    -------
    (call_price, delta): tuple of floats
        call_price : The Black–Scholes price of the call.
        delta      : The partial derivative of the call price wrt. the stock price.
    """
    # d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Call price
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    # Delta
    delta_call = norm.cdf(d1)

    return call_price, delta_call

def delta_hedged_short_call_example(S, K, T, r, sigma):
    """
    Demonstrates a delta-hedged short call using:
     - The short call option
     - A delta hedge in the underlying
     - A risk-free borrowing/lending position

    It sets up the position, so the initial net cash flow is zero,
    then calculates the final P&L at two potential stock prices at maturity
    and ignoring continuous balancing.
    """

    # 1) Compute call price and delta using Black–Scholes
    call_price, delta_call = black_scholes_call(S, K, T, r, sigma)

    print("=== Black–Scholes Computations ===")
    print(f"Underlying price (S0) = {S:.2f}")
    print(f"Strike (K)            = {K:.2f}")
    print(f"Time to maturity (T)  = {T:.2f} year(s)")
    print(f"Risk-free rate (r)    = {r:.2%}")
    print(f"Volatility (sigma)    = {sigma:.2%}\n")
    print(f"Call Price = ${call_price:.2f}")
    print(f"Call Delta = {delta_call:.3f}\n")

    # 2) Set up the delta-hedged short call

    # Short 1 call => collect premium
    premium_received = call_price

    # Buy delta_call shares => negative cashflow
    cost_shares = delta_call * S

    # Borrow the difference so initial net = 0
    borrowed = cost_shares - premium_received

    print("=== Position Setup (Initial) ===")
    print(f"Short 1 call: +${premium_received:.2f} premium")
    print(f"Buy {delta_call:.3f} shares: -${cost_shares:.2f}")
    print(f"Borrow cash: +${borrowed:.2f} (to net initial cashflow to $0.00)\n")

    # 3) Evaluate final payoff for two terminal prices
    #    ignoring continuous rebalancing

    terminal_prices = [90.0, 120.0]

    print("=== Payoff at Expiry (Discrete, No Rebalancing or Interest) ===")
    for S_T in terminal_prices:
        # Short call payoff: -(max(S_T - K, 0)) #plus the premium we initially collected
        call_exercise_value = max(S_T - K, 0.0)
        short_call_pnl = premium_received - call_exercise_value
        # short_call_pnl = call_exercise_value


        # Stock payoff: final stock value minus what we paid at inception
        stock_final_value = delta_call * S_T
        stock_pnl = stock_final_value - cost_shares

        # Bond/cash payoff: we owe exactly the borrowed amount
        # (ignoring interest)
        bond_pnl = borrowed - borrowed * (1+r)**T

        total_pnl = short_call_pnl + stock_pnl + bond_pnl

        print(f"Final Stock Price: S_T = ${S_T:.2f}")
        print(f"  - Short Call P&L  = ${short_call_pnl:.2f}")
        print(f"  - Stock P&L       = ${stock_pnl:.2f}")
        print(f"  - Bond (Borrowed) = ${bond_pnl:.2f}")
        print(f"  --> TOTAL P&L     = ${total_pnl:.2f}\n")


delta_hedged_short_call_example(
    S=100.0,
    K=100.0,
    T=1.0,
    r=0.05,
    sigma=0.25
)


=== Black–Scholes Computations ===
Underlying price (S0) = 100.00
Strike (K)            = 100.00
Time to maturity (T)  = 1.00 year(s)
Risk-free rate (r)    = 5.00%
Volatility (sigma)    = 25.00%

Call Price = $12.34
Call Delta = 0.627

=== Position Setup (Initial) ===
Short 1 call: +$12.34 premium
Buy 0.627 shares: -$62.74
Borrow cash: +$50.40 (to net initial cashflow to $0.00)

=== Payoff at Expiry (Discrete, No Rebalancing or Interest) ===
Final Stock Price: S_T = $90.00
  - Short Call P&L  = $12.34
  - Stock P&L       = $-6.27
  - Bond (Borrowed) = $-2.52
  --> TOTAL P&L     = $3.54

Final Stock Price: S_T = $120.00
  - Short Call P&L  = $-7.66
  - Stock P&L       = $12.55
  - Bond (Borrowed) = $-2.52
  --> TOTAL P&L     = $2.36

